In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
file = "Data\\fivepoints.csv"
df = pd.read_csv(file, sep=';', low_memory=False, encoding='ISO-8859-1')

In [5]:
if df['primary'].nunique() == df.shape[0]:
    print("All values in 'primary' are different.")

All values in 'primary' are different.


In [6]:
if df['idll_vo3lv'].nunique() == df.shape[0]:
    print("All values in 'student id' are different.")
else:
    print("no")

no


In [7]:
df.dtypes

primary        int64
idll_vo3lv     int64
name          object
value         object
dtype: object

Primary Key: unique identified for a single test
idll_vo3lv: student ID

In [8]:
student_ids = df['idll_vo3lv'].unique()

In [9]:
df['name'].unique()

array(['patternsm', 'timestampsm', 'statuscl', 'numbercl', 'timestampcl',
       'timestampload'], dtype=object)

In [10]:
sorted_df = df.sort_values(by=['idll_vo3lv'], ascending=True)

In [11]:
restructured_columns = {
    'student_id': [],
    'patternsm': [],
    'statuscl': [],
    'numbercl': [],
    'timestampcl': [],
    'timestampload': [],
    'timestampsm': [],
    'primary_keys': []
}

for studentid in student_ids:
    results = df[df['idll_vo3lv'] == studentid]
    sorted_results = results.sort_values(by=['primary'], ascending=True)

    # Values
    primary_keys = []
    patternsm = None
    statuscl = None
    numbercl = None
    timestampcl = None
    timestampload = None
    timestampsm = None

    
    i = 0 

    for _, row in sorted_results.iterrows():

        if row['name'] == 'patternsm':
            patternsm = row['value']
        elif row['name'] == 'statuscl':
            statuscl = row['value']
        elif row['name'] == 'numbercl':
            numbercl = row['value']
        elif row['name'] == 'timestampcl':
            timestampcl = row['value']
        elif row['name'] == 'timestampload':
            timestampload = row['value']
        elif row['name'] == 'timestampsm':
            timestampsm = row['value']
        else: 
            print(f"{row['primary']} has {row['name']}, and it failed")

        # When complete, add the values to the column
        if i == 5:
            restructured_columns['student_id'].append(studentid)
            restructured_columns['patternsm'].append(patternsm)
            restructured_columns['statuscl'].append(statuscl)
            restructured_columns['numbercl'].append(numbercl)
            restructured_columns['timestampcl'].append(timestampcl)
            restructured_columns['timestampload'].append(timestampload)
            restructured_columns['timestampsm'].append(timestampsm)
            restructured_columns['primary_keys'].append(primary_keys)
            i == 0
        
        primary_keys.append(row['primary'])
        i += 1


restructured_df = pd.DataFrame(restructured_columns)
restructured_df.to_csv("Data\sortedfivepointtest.csv")